# Python Core

In this file, we will review intermediate Python concepts. This chapter is intended for readers who already have a little programming experience in Python, we assume you're beginning to be comfortable with the basics such as variables, loops, and functions, classes, object-oriented-programming. We will focus on concepts that are less trivial but highly relevant, especially in professional environments and technical interviews for Python or finance-related roles.

The following notions will be discussed : arguments, isinstance, comprehensions, iterators & generators, decorators, OOP, functionnal programming tools, exceptions handling, context managers, typing
__________________________________________________________________________________________________

### Arguments

Python has two types of arguments: positional arguments and keyword arguments, often denoted as `*args` and `**kwargs`.

- `*args` This notation is used to pass a variable number of positional argu-
ments to a function. It collects any extra unnamed arguments into a tuple.
This is useful when you’re not sure how many inputs your function will receive.

- `**kwargs` allows a function to accept any number of keyword arguments (i.e., named arguments). These are collected into a dictionary.

In [66]:
def sum_all(*args):
    return sum(args)

print("Args sum: ", sum_all(1, 2, 3))

def print_info(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

print_info(name="Alice", age=30)

Args sum:  6
name: Alice
age: 30


### Isinstance

The `isinstance()` function in Python is used to check if an object is an
instance of a specified class or a tuple of classes. It returns True if the object
matches the given type(s), and False otherwise. This function is particularly
useful for type checking and ensuring that variables have the expected data type
before performing operations on them, which helps to avoid runtime errors.


In [42]:
x = 10
print(isinstance(x, int))
print(isinstance(x, (int, float)))
print(isinstance(x, str))

True
True
False


In [43]:
class Animal:
    pass

class Dog(Animal):
    pass

dog = Dog()
print(isinstance(dog, Dog))
print(isinstance(dog, Animal))

True
True


### Comprehensions

Comprehensions provide a concise and readable way to create collections such as lists, dictionaries, and sets by embedding a loop and optional conditional logic inside a single expression. They generally allow clearer and more efficient code compared to traditional loops. Closely related are generator expressions, which create iterators that generate items lazily, making them memory-efficient for large datasets.

List comprehensions generate lists:

In [44]:
squares = [x**2 for x in range(10) if x % 2 == 0]
print(squares)

[0, 4, 16, 36, 64]


Dict comprehensions build dictionaries similarly:


In [45]:
square_dict = {x: x**2 for x in range(5)}
print(square_dict)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}


Note that overly complex comprehensions can harm readability and maintainability. A regular loop can be clearer.

In [46]:
result = [str(x**2) if x % 2 == 0 else f"Odd:{x}" for x in range(10) if x > 2 and x != 5]
print(result)

['Odd:3', '16', '36', 'Odd:7', '64', 'Odd:9']


### Iterators and Generators

Python uses the iterators to allow objects to be traversed one element at a time, enabling efficient looping over large or complex data structures. Generators are a convenient way to create iterators using the `yield` keyword, which produces values lazily, pausing and resuming execution as needed. This makes generators ideal for data pipelines and lazy evaluation, where processing large datasets incrementally reduces memory consumption.

*The Iterator Protocol* requires an object to implement `__iter__()` returning an iterator, and `__next__()` to get the next item or raise `StopIteration`.

In [47]:
class CountUpTo:
    def __init__(self, max):
        self.max = max
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current >= self.max:
            raise StopIteration
        self.current += 1
        return self.current

counter = CountUpTo(3)
for number in counter:
    print(number)

1
2
3


Creating generators with `yield` simplifies iterator creation by automatically handling state. These concepts are especially useful in data pipelines, where generators allow processing large data streams incrementally without loading everything into memory.


In [48]:
def count_up_to(max):
    current = 1
    while current <= max:
        yield current
        current += 1

for number in count_up_to(3):
    print(number)

1
2
3


### Decorators

Functions are first-class objects, which means they can be assigned to vari-
ables, passed as arguments, and returned from other functions. This feature
enables decorators, which are functions that take another function and extend
or modify its behavior without changing its source code.


In [49]:
def greet():
    print("Hello!")

say_hello = greet
say_hello()

Hello!


In [50]:
def decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@decorator
def greet():
    print("Hello!")

greet()

Before function call
Hello!
After function call


Let’s write a practical example in the following context: Maybe functions
are implement in an application I built, and I would like to print the runtime of
function calls. One solution consists in modifying each function I implemented
to do se. Another solution is to implement a Timer decorator.

In [51]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        runtime = end - start
        print(f"{func.__name__} took {runtime:.4f} seconds")
        return result
    return wrapper

@timer
def waste_time():
    for _ in range(10**6):
        pass

waste_time()

waste_time took 0.0708 seconds


### Object-Oriented Programming

OOP in Python enables structuring code using classes and objects. It sup-
ports concepts like inheritance, encapsulation, and polymorphism to model real-
world entities.


In [52]:
class Animal:
    def speak(self):
        print("Some generic sound")

class Dog(Animal):
    def speak(self):
        print("Woof!")

dog = Dog()
dog.speak()

Woof!


Special methods allow customizing class behavior. For example, `__init__` initializes objects, and `__repr__` defines the official string representation.

In [53]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

p = Person("Alice", 30)
print(p)

Person(name=Alice, age=30)


Instance variables belong to individual objects, while class variables are
shared among all instances of a class.

In [54]:
class Car:
    wheels = 4  # class variable

    def __init__(self, color):
        self.color = color  # instance variable

car1 = Car("red")
car2 = Car("blue")

print("Car 1 attributes: ", car1.wheels, car1.color)
print("Car 2 attributes: ", car2.wheels, car2.color)

Car.wheels = 3

print("Car1 wheels: ", car1.wheels)
print("Car2 wheels: ", car2.wheels)

Car 1 attributes:  4 red
Car 2 attributes:  4 blue
Car1 wheels:  3
Car2 wheels:  3


In Python, the built-in function `dir()` allows you to list the attributes and methods of any object. This is useful to explore what capabilities an object has, including methods inherited from parent classes.

For example, all classes in Python implicitly inherit from the base `object` class, which provides fundamental methods such as `__init__`, `__repr__`, `__eq__`, and `__hash__`.

In [55]:
print(dir(object))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


These methods define basic behavior for all Python objects, such as object
creation, comparison, string representation, and attribute access. I highly en-
courage you to play with those methods, try to re-implement them and figure
out as precisely as possible what their functioning is.

### Functional Programming Tools

Python supports several functional programming constructs that allow concise and expressive manipulation of data.

**map, filter, reduce**
- `map` applies a function to all items in an iterable.
- `filter` selects items from an iterable based on a predicate function.
- `reduce` (from `functools`) aggregates items in an iterable using a binary function.

In [56]:
from functools import reduce

nums = [1, 2, 3, 4]

squared = list(map(lambda x: x**2, nums))
evens = list(filter(lambda x: x % 2 == 0, nums))
sum_all = reduce(lambda a, b: a + b, nums)

print(squared)
print(evens)
print(sum_all)

[1, 4, 9, 16]
[2, 4]
10


Lambda functions are anonymous, inline functions useful for short, simple
operations.

In [57]:
add = lambda x, y: x + y
print(add(3, 5))

8


The `functools` module provides higher-order functions like:  
- **lru_cache**: caches results of expensive function calls to optimize repeated calls.  
- **partial**: creates a new function with some arguments fixed.

In [58]:
from functools import lru_cache, partial

@lru_cache(maxsize=None)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

add_five = partial(lambda x, y: x + y, 5)
print(fibonacci(10))
print(add_five(10))

55
15


### Exception Handling

Python provides a robust mechanism to handle runtime errors through `try`/`except` blocks, allowing programs to recover gracefully or clean up resources, using four essential keywords: `try`, `except`, `else`, `finally`.

- `try` block contains code that might raise exceptions.
- `except` catches and handles exceptions.
- `else` executes if no exceptions occur.
- `finally` always executes, typically used for cleanup.


In [59]:
try:
    result = 10 / 0
except ZeroDivisionError:
    print("Cannot divide by zero.")
else:
    print(f"Result is {result}")
finally:
    print("Execution completed.")

Cannot divide by zero.
Execution completed.


You can define your own exception classes by subclassing Exception to
represent specific error conditions.

In [60]:
class InsufficientFunds(Exception):
    pass

def withdraw(amount, balance):
    if amount > balance:
        raise InsufficientFunds("Not enough balance")
    return balance - amount

try:
    withdraw(100, 50)
except InsufficientFunds as e:
    print(e)

Not enough balance


Note that a good practice is to catch specific exceptions rather than using
bare except.

### Context Managers

Context managers in Python simplify resource management by ensuring setup and cleanup actions are executed automatically, typically using the `with` statement.  
The `with` statement automatically manages resources like files, closing them when the block ends, even if exceptions occur. Resource closing is **not automatic** when used out of a `with` clause.


In [61]:
# That won't work if we have no 'data.txt' file in the directory but you got the idea...

with open("data.txt", "r") as file:
    contents = file.read()

print(file.closed)

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'

### Typing and Type Hints

Python supports optional type hints to improve code clarity and enable static type checking without affecting runtime behavior.

You can annotate function parameters and return types using colon and arrow syntax.


In [71]:
def greet(name: str) -> str:
    return f"Hello, {name}"

The `typing` module provides advanced types like `Union` for multiple possible types, `Optional` for nullable values, and `Callable` for function signatures.


In [72]:
from typing import Union, Optional, Callable

def process(value: Union[int, str]) -> Optional[str]:
    if isinstance(value, int):
        return str(value)
    elif isinstance(value, str):
        return value.upper()
    return None

def executor(func: Callable[[int, int], int], a: int, b: int) -> int:
    return func(a, b)

print(process(42))           # Output: 42
print(process("hello"))      # Output: HELLO
print(executor(lambda x, y: x + y, 2, 3))  # Output: 5


42
HELLO
5


Tools like `mypy` analyze type hints to detect potential bugs before runtime, improving code reliability and maintainability.

**Note that using type hints is not mandatory**, and will not cause any exceptions to be raised; it just serves an indicative purpose.
